### Monthly streamflow indicators computation considering natural versus modified conditions:

#### This code provide the automatic calculation of the Monthly Hydrological Indicators for river streamflow. It follows the methodology proposed by Pumo et al. (2018), which is an addaptation of the methodlogy proposed for daily streamflow by Richter et al. (1996).

In total there are 22 individual indicators, 5 group indices (MI-HRA), and a Global indice (GMI-HRA).

References: Pumo, D., Francipane, A., Cannarozzo, M., Antinoro, C., Noto, L.V., 2018. Monthly hydrological indicators to assess possible alterations on rivers' flow regime. Water Resour. Manag.
32, 3687–3706. https://doi.org/10.1007/s11269-018-2013-6.

Richter, B.D., Baumgartner, J.V., Powell, J., Braun, D.P., 1996. A method for assessing hydrologic alteration within ecosystems. Conserv. Biol. 10, 1163–1174. https://doi.org/10.1046/j.1523-1739.1996.10041163.x.

Developed by: Thiago Victor Medeiros do Nascimento

##### Observations: 

You will need only two files: 
1. One xlsx with the natural streamflows organized with an index called preferable date with the dates, and each column representing one different station/sub-basin to be analyed.
1. Similarly, one xlsx with the modified streamflows organized with an index called preferable date with the dates, and each column representing one different station/sub-basin to be analyed.

##### In this example you have a only one station, but the code is ready for more. 

In [1]:
import numpy as np
import os
import pandas as pd
import datetime as dt

In [2]:
# Set and print your working directory

os.chdir(r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicators')
print("Current Working Directory " , os.getcwd())

Current Working Directory  C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicators


In [3]:
# Firstly one function is defined for the computation of the generic k-th indicator of hydrological alteration:
def pik(Xn25ik, Xn75ik, Xpik):
    if (Xpik >= Xn25ik) and (Xpik <= Xn75ik):
        result = 0
    else:
        result = min(abs((Xpik - Xn25ik)/(Xn75ik - Xn25ik)), abs((Xpik - Xn75ik)/(Xn75ik - Xn25ik)))   
    return result

In [4]:
# Read the xlsx file with your nonfilled data. This example provides data for four sub-basins and already separated in natural and modified conditions:
path =r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicators\cardeiratesenatural.xlsx'
datanatural=pd.read_excel(path)
datanatural.set_index(datanatural.date, inplace=True)
datanatural["month"] = datanatural.date.dt.month
datanatural.drop(columns=['date'], inplace = True)  
datanatural = datanatural.loc['10-01-1934':]

path =r'C:\Users\User\OneDrive\IST\RESEARCH\python\flowindicators\cardeiratesemodificado.xlsx'
datamodified=pd.read_excel(path)
datamodified.set_index(datamodified.date, inplace=True)
datamodified["month"] = datamodified.date.dt.month
datamodified.drop(columns=['date'], inplace = True)  

In [5]:
datanatural.head()

,cardeiranatural,month
date,,
1934-10-31,0.0,10
1934-11-30,0.0,11
1934-12-31,0.0,12
1935-01-31,0.0,1
1935-02-28,0.0,2


In [6]:
datamodified.head()

,cardeiramodificado,month
date,,
2000-10-31,0.000000,10
2000-11-30,0.005653,11
2000-12-31,84.756966,12
2001-01-31,52.084089,1
2001-02-28,50.397121,2


In [7]:
# The number of sub-basins (stations) is computed:
numstationsused = datanatural.shape[1]-1
print("The number of stations used is: ", numstationsused)

The number of stations used is:  1


### Individual indicators computation for each statistical group:

#### Magnitude timing (Group 1):

In [8]:
# Statistics (median and quantiles):
MedianMonthlyStreamflow = datamodified.groupby('month',dropna=False).median()
Quantile25MonthlyStreamflow = datanatural.groupby('month',dropna=False).quantile(q=0.25)
Quantile75MonthlyStreamflow = datanatural.groupby('month',dropna=False).quantile(q=0.75)

Quantile75MonthlyStreamflow

,cardeiranatural
month,
1,41.630130
2,31.611123
3,34.406334
4,6.961309
5,0.018896
6,0.000000
7,0.000000
8,0.000000
9,0.000000


In [9]:
# This is an empty table to be filled with the indicators for group 1:
p1s = pd.DataFrame(index = Quantile75MonthlyStreamflow.index, columns = Quantile75MonthlyStreamflow.columns, data=np.zeros((12,numstationsused)))

# The computation is made for each station (sub-basin):
j = 0 
for numstations in range(datanatural.shape[1]-1):
    for k in range(12):
        p1s.iloc[k,j] = pik(Quantile25MonthlyStreamflow.iloc[k,j], Quantile75MonthlyStreamflow.iloc[k,j], MedianMonthlyStreamflow.iloc[k,j])
    j = j + 1

In [10]:
p1s

,cardeiranatural
month,
1,7.941688e-04
2,3.091875e-03
3,9.900915e-07
4,0.000000e+00
5,0.000000e+00
6,0.000000e+00
7,0.000000e+00
8,0.000000e+00
9,0.000000e+00


In [11]:
# Computation of the Group index (Group 1):
MIhra1 = p1s.mean()
MIhra1

cardeiranatural    0.000324
dtype: float64

#### Magnitude duration (Group 2):

In [12]:
# Statistics for 3-months:
Months3flownat = datanatural.iloc[:,0:-1].resample('3M',closed='left').sum()
Months3flowmod = datamodified.iloc[:,0:-1].resample('3M',closed='left').sum()

# Computation of the water years for natural conditions:
Months3flownat["datetime"] = Months3flownat.index
Months3flownat['water_year'] = Months3flownat.datetime.dt.year.where(Months3flownat.datetime.dt.month < 10, Months3flownat.datetime.dt.year + 1)
# Correction of a small bug in the resample:
Months3flownat['water_year'] = Months3flownat.water_year.where(Months3flownat.datetime.dt.month != 10, Months3flownat.water_year - 1)
Months3flownat.drop(columns=['datetime'], inplace = True)

# Computation of the water years for modified conditions:
Months3flowmod["datetime"] = Months3flowmod.index
Months3flowmod['water_year'] = Months3flowmod.datetime.dt.year.where(Months3flowmod.datetime.dt.month < 10, Months3flowmod.datetime.dt.year + 1)
# Correction of a small bug in the resample:
Months3flowmod['water_year'] = Months3flowmod.water_year.where(Months3flowmod.datetime.dt.month != 10, Months3flowmod.water_year - 1)
Months3flowmod.drop(columns=['datetime'], inplace = True)

# Minimum and maximum for 3-months:
AnnualMin3MonthsFlownat = Months3flownat.groupby('water_year',dropna=False).min()
AnnualMax3MonthsFlownat = Months3flownat.groupby('water_year',dropna=False).max()

AnnualMin3MonthsFlowmod = Months3flowmod.groupby('water_year',dropna=False).min()
AnnualMax3MonthsFlowmod = Months3flowmod.groupby('water_year',dropna=False).max()

In [13]:
# Statistics for 6-months:
Months6flownat = datanatural.iloc[:,0:-1].resample('6M',closed='left').sum()
Months6flowmod = datamodified.iloc[:,0:-1].resample('6M',closed='left').sum()

# Computation of the water years for natural conditions:
Months6flownat["datetime"] = Months6flownat.index
Months6flownat['water_year'] = Months6flownat.datetime.dt.year.where(Months6flownat.datetime.dt.month < 10, Months6flownat.datetime.dt.year + 1)
# Correction of a small bug in the resample:
Months6flownat['water_year'] = Months6flownat.water_year.where(Months6flownat.datetime.dt.month != 10, Months6flownat.water_year - 1)
Months6flownat.drop(columns=['datetime'], inplace = True)

# Computation of the water years for modified conditions:
Months6flowmod["datetime"] = Months6flowmod.index
Months6flowmod['water_year'] = Months6flowmod.datetime.dt.year.where(Months6flowmod.datetime.dt.month < 10, Months6flowmod.datetime.dt.year + 1)
# Correction of a small bug in the resample:
Months6flowmod['water_year'] = Months6flowmod.water_year.where(Months6flowmod.datetime.dt.month != 10, Months6flowmod.water_year - 1)
Months6flowmod.drop(columns=['datetime'], inplace = True)

# Minimum and maximum for 6-months:
AnnualMin6MonthsFlownat = Months6flownat.groupby('water_year',dropna=False).min()
AnnualMax6MonthsFlownat = Months6flownat.groupby('water_year',dropna=False).max()

AnnualMin6MonthsFlowmod = Months6flowmod.groupby('water_year',dropna=False).min()
AnnualMax6MonthsFlowmod = Months6flowmod.groupby('water_year',dropna=False).max()

In [14]:
# Computation of the median and quantiles:
## First empty data frames for each case are made:
MedianAnnualMinMax2 = pd.DataFrame(index = ["Min3months","Max3months","Min6months","Max6months"], columns = AnnualMax6MonthsFlownat.columns)
Quantile25AnnualMinMax2 = pd.DataFrame(index = ["Min3months","Max3months","Min6months","Max6months"], columns = AnnualMax6MonthsFlownat.columns)
Quantile75AnnualMinMax2 = pd.DataFrame(index = ["Min3months","Max3months","Min6months","Max6months"], columns = AnnualMax6MonthsFlownat.columns)

# The median is computed only for the modified streamflow:
MedianAnnualMinMax2.iloc[0,:] = AnnualMin3MonthsFlowmod.median()
MedianAnnualMinMax2.iloc[1,:] = AnnualMax3MonthsFlowmod.median()
MedianAnnualMinMax2.iloc[2,:] = AnnualMin6MonthsFlowmod.median()
MedianAnnualMinMax2.iloc[3,:] = AnnualMax6MonthsFlowmod.median()

# The quantiles are computed only for the natural streamflow:
Quantile25AnnualMinMax2.iloc[0,:] = AnnualMin3MonthsFlownat.quantile(q=0.25)
Quantile25AnnualMinMax2.iloc[1,:] = AnnualMax3MonthsFlownat.quantile(q=0.25)
Quantile25AnnualMinMax2.iloc[2,:] = AnnualMin6MonthsFlownat.quantile(q=0.25)
Quantile25AnnualMinMax2.iloc[3,:] = AnnualMax6MonthsFlownat.quantile(q=0.25)

Quantile75AnnualMinMax2.iloc[0,:] = AnnualMin3MonthsFlownat.quantile(q=0.75)
Quantile75AnnualMinMax2.iloc[1,:] = AnnualMax3MonthsFlownat.quantile(q=0.75)
Quantile75AnnualMinMax2.iloc[2,:] = AnnualMin6MonthsFlownat.quantile(q=0.75)
Quantile75AnnualMinMax2.iloc[3,:] = AnnualMax6MonthsFlownat.quantile(q=0.75)

In [15]:
# This is an empty table to be filled with the indicators for group 2:
p2s = pd.DataFrame(index = Quantile75AnnualMinMax2.index, columns = Quantile75MonthlyStreamflow.columns, data=np.nan)

# Loop for computing for each station:
j = 0 
for numstations in range(numstationsused):
    for k in range(4):
        p2s.iloc[k,j] = pik(Quantile25AnnualMinMax2.iloc[k,j], Quantile75AnnualMinMax2.iloc[k,j], MedianAnnualMinMax2.iloc[k,j])
    j = j + 1

In [16]:
p2s

,cardeiranatural
Min3months,0.000000
Max3months,0.167737
Min6months,0.000039
Max6months,0.128157


In [17]:
# Computation of the Group index (Group 2):
MIhra2 = p2s.mean()
MIhra2

cardeiranatural    0.073983
dtype: float64

#### Timing (Group 3):

In [18]:
# Cliping the data to be used:
datanatural3 = pd.DataFrame(data= datanatural.drop(columns = "month"))
datamodified3 = pd.DataFrame(data= datamodified.drop(columns = "month"))

# Computation of the water years for natural condition:
datanatural3["datetime"] = datanatural3.index
datanatural3['water_year'] = datanatural3.datetime.dt.year.where(datanatural3.datetime.dt.month < 10, datanatural3.datetime.dt.year + 1)
datanatural3.drop(columns=['datetime'], inplace = True)

# Computation of the water years for modified condition:
datamodified3["datetime"] = datamodified3.index
datamodified3['water_year'] = datamodified3.datetime.dt.year.where(datamodified3.datetime.dt.month < 10, datamodified3.datetime.dt.year + 1)
datamodified3.drop(columns=['datetime'], inplace = True)

# The ID (location) of each minimum or maximum is computed:
Mininumslocationnatural = datanatural3.groupby('water_year',dropna=False).idxmin()
Maximumslocationnatural = datanatural3.groupby('water_year',dropna=False).idxmax()
Mininumslocationmodified = datamodified3.groupby('water_year',dropna=False).idxmin()
Maximumslocationmodified= datamodified3.groupby('water_year',dropna=False).idxmax()

In [19]:
# Empty tables to be filled with the actual month of each extreme are built:
# Natural:
MininumslocationMonthnatural = pd.DataFrame(index = Mininumslocationnatural.index, columns = Mininumslocationnatural.columns)
MaximumlocationMonthnatural = pd.DataFrame(index = Mininumslocationnatural.index, columns = Mininumslocationnatural.columns)

# Modified:
MininumslocationMonthmodified = pd.DataFrame(index = Mininumslocationmodified.index, columns = Mininumslocationmodified.columns)
MaximumlocationMonthmodified = pd.DataFrame(index = Mininumslocationmodified.index, columns = Mininumslocationmodified.columns)

# The month of each specific event (maximum and minimum) is computed:
for i in range(numstationsused):
    MininumslocationMonthnatural.iloc[:,i] = Mininumslocationnatural.iloc[:,i].dt.month
    MaximumlocationMonthnatural.iloc[:,i] = Maximumslocationnatural.iloc[:,i].dt.month
    MininumslocationMonthmodified.iloc[:,i] = Mininumslocationmodified.iloc[:,i].dt.month
    MaximumlocationMonthmodified.iloc[:,i] = Maximumslocationmodified.iloc[:,i].dt.month


# Replace the months according to the classification of the paper used: (May = 1 until April = 12)
MininumslocationMonthnatural.replace([5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace = True)
MaximumlocationMonthnatural.replace([5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace = True)
MininumslocationMonthmodified.replace([5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace = True)
MaximumlocationMonthmodified.replace([5, 6, 7, 8, 9, 10, 11, 12, 1, 2, 3, 4], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], inplace = True)

In [20]:
# Computation of the median and quantiles:
## First empty data frames for each case are made:
MedianMonths = pd.DataFrame(index = ["Min","Max"], columns = MininumslocationMonthmodified.columns)
Quantile25Months = pd.DataFrame(index = ["Min","Max"], columns = MininumslocationMonthnatural.columns)
Quantile75Months = pd.DataFrame(index = ["Min","Max"], columns = MininumslocationMonthnatural.columns)

# The median(mode) is computed only for the modified streamflow:
MedianMonths.iloc[0,:] = MininumslocationMonthmodified.mode(dropna=False).max()
MedianMonths.iloc[1,:] = MaximumlocationMonthmodified.mode(dropna=False).max()


# The quantiles are computed only for the natural streamflow:
Quantile25Months.iloc[0,:] = MininumslocationMonthnatural.quantile(q=0.25)
Quantile25Months.iloc[1,:] = MaximumlocationMonthnatural.quantile(q=0.25)

Quantile75Months.iloc[0,:] = MininumslocationMonthnatural.quantile(q=0.75)
Quantile75Months.iloc[1,:] = MaximumlocationMonthnatural.quantile(q=0.75)

In [21]:
# This is an empty table to be filled with the indicators for group 3:
p3s = pd.DataFrame(index = Quantile25Months.index, columns = Quantile25Months.columns, data=np.nan)

# Loop for computing for each station:
j = 0 
for numstations in range(numstationsused):
    for k in range(2):
        p3s.iloc[k,j] = pik(Quantile25Months.iloc[k,j], Quantile75Months.iloc[k,j], MedianMonths.iloc[k,j])
    j = j + 1

In [22]:
p3s

,cardeiranatural
Min,0.000000
Max,0.666667


In [23]:
# Computation of the Group index (Group 5):
MIhra3 = p3s.mean()
MIhra3

cardeiranatural    0.333333
dtype: float64

#### Magnitude frequency (Group 4):

In [24]:
# The quantiles 10% and 90% are computed:
Quantile10Streamflow = datanatural.iloc[:,0:-1].quantile(q=0.10)
Quantile90Streamflow = datanatural.iloc[:,0:-1].quantile(q=0.90)

datanatural4 = datanatural.drop(columns = "month")
datamodified4 = datamodified.drop(columns = "month")

# Creating empty tables for data filling:
condlowpulsesnat = pd.DataFrame(index = datanatural4.index, columns = datanatural4.columns, data=np.nan)
condhighpulsesnat = pd.DataFrame(index = datanatural4.index, columns = datanatural4.columns, data=np.nan)
condlowpulsesmod = pd.DataFrame(index = datamodified4.index, columns = datamodified4.columns, data=np.nan)
condhighpulsesmod = pd.DataFrame(index = datamodified4.index, columns = datamodified4.columns, data=np.nan)

for i in range(numstationsused):
    condlowpulsesnat.iloc[:,i] = np.where((datanatural4.iloc[:,i] < Quantile10Streamflow[i]),1,0)
    condhighpulsesnat.iloc[:,i] = np.where((datanatural4.iloc[:,i] > Quantile90Streamflow[i]),1,0)
    condlowpulsesmod.iloc[:,i] = np.where((datamodified4.iloc[:,i] < Quantile10Streamflow[i]),1,0)
    condhighpulsesmod.iloc[:,i] = np.where((datamodified4.iloc[:,i] > Quantile90Streamflow[i]),1,0)

# Computing the number of months per year:
condlowpulsesnat["datetime"] = condlowpulsesnat.index

# Computation of water-years:
condlowpulsesnat['year'] = condlowpulsesnat.datetime.dt.year.where(condlowpulsesnat.datetime.dt.month < 10, condlowpulsesnat.datetime.dt.year + 1)
condhighpulsesnat['year'] = condlowpulsesnat['year']
condlowpulsesmod['year'] = condlowpulsesnat['year']
condhighpulsesmod['year'] = condlowpulsesnat['year']

condlowpulsesnat.drop(columns=['datetime'], inplace = True)  

In [25]:
# The total number of low and high pulses are computed for each situation:
lowpulsesnat = condlowpulsesnat.groupby('year',dropna=False).sum()
highpulsesnat = condhighpulsesnat.groupby('year',dropna=False).sum()
lowpulsesmod = condlowpulsesmod.groupby('year',dropna=False).sum()
highpulsesmod = condhighpulsesmod.groupby('year',dropna=False).sum()

In [26]:
# Computation of the median and quantiles:
## First empty data frames for each case are made:
MedianLowAndHighPulses = pd.DataFrame(index = ["lowpulses","highpulses"], columns = lowpulsesmod.columns)
Quantile25LowAndHighPulses = pd.DataFrame(index = ["lowpulses","highpulses"], columns = lowpulsesnat.columns)
Quantile75LowAndHighPulses = pd.DataFrame(index = ["lowpulses","highpulses"], columns = lowpulsesnat.columns)

# The median is computed only for the modified streamflow:
MedianLowAndHighPulses.iloc[0,:] = lowpulsesmod.median()
MedianLowAndHighPulses.iloc[1,:] = highpulsesmod.median()


# The quantiles are computed only for the natural streamflow:
Quantile25LowAndHighPulses.iloc[0,:] = lowpulsesnat.quantile(q=0.25)
Quantile25LowAndHighPulses.iloc[1,:] = highpulsesnat.quantile(q=0.25)

Quantile75LowAndHighPulses.iloc[0,:] = lowpulsesnat.quantile(q=0.75)
Quantile75LowAndHighPulses.iloc[1,:] = highpulsesnat.quantile(q=0.75)

In [27]:
# This is an empty table to be filled with the indicators for group 4:
p4s = pd.DataFrame(index = Quantile25LowAndHighPulses.index, columns = Quantile25LowAndHighPulses.columns, data=np.nan)

# Loop for computing for each station:
j = 0 
for numstations in range(numstationsused):
    for k in range(2):
        p4s.iloc[k,j] = pik(Quantile25LowAndHighPulses.iloc[k,j], Quantile75LowAndHighPulses.iloc[k,j], MedianLowAndHighPulses.iloc[k,j])
    j = j + 1

In [28]:
p4s

,cardeiranatural
lowpulses,0.0
highpulses,2.0


In [29]:
# Computation of the Group index (Group 5):
MIhra4 = p4s.mean()
MIhra4

cardeiranatural    1.0
dtype: float64

#### Frequency rate of change (Group 5):

In [30]:
# Statistics:
# Obtain the data without the month within:
datanatural5 = datanatural.drop(columns = "month")
datamodified5 = datamodified.drop(columns = "month")
# Cumulative differences: 
diffnatural = datanatural5.diff(1)
diffmodififed = datamodified5.diff(1)

# Compute separatly the positive and the negative differences:
diffnaturalpositives = diffnatural[diffnatural>=0]
diffnaturalnegatives = diffnatural[diffnatural<0]

diffmodifiedpositives = diffmodififed[diffmodififed>=0]
diffmodifiednegatives = diffmodififed[diffmodififed<0]

# Computation of the median and quantiles:
## First empty data frames for each case are made:
MedianDifferences = pd.DataFrame(index = ["positives","negatives"], columns = diffmodifiednegatives.columns)
Quantile25Differences = pd.DataFrame(index = ["positives","negatives"], columns = diffnaturalpositives.columns)
Quantile75Differences = pd.DataFrame(index = ["positives","negatives"], columns = diffnaturalpositives.columns)

# The median is computed only for the modified streamflow:
MedianDifferences.iloc[0,:] = diffmodifiedpositives.median(skipna=True)
MedianDifferences.iloc[1,:] = diffmodifiednegatives.median(skipna=True)

# The quantiles are computed only for the natural streamflow:
Quantile25Differences.iloc[0,:] = diffnaturalpositives.quantile(q=0.25)
Quantile25Differences.iloc[1,:] = diffnaturalnegatives.quantile(q=0.25)

Quantile75Differences.iloc[0,:] = diffnaturalpositives.quantile(q=0.75)
Quantile75Differences.iloc[1,:] = diffnaturalnegatives.quantile(q=0.75)

In [31]:
# This is an empty table to be filled with the indicators for group 5:
p5s = pd.DataFrame(index = Quantile25Differences.index, columns = Quantile25Differences.columns, data=np.nan)

# Loop for computing for each station:
j = 0 
for numstations in range(numstationsused):
    for k in range(2):
        p5s.iloc[k,j] = pik(Quantile25Differences.iloc[k,j], Quantile75Differences.iloc[k,j], MedianDifferences.iloc[k,j])
    j = j + 1

In [32]:
p5s

,cardeiranatural
positives,0.0
negatives,0.0


In [33]:
# Computation of the Group index (Group 5):
MIhra5 = p5s.mean()
MIhra5

cardeiranatural    0.0
dtype: float64

#### Concatetanion of all the group indices in one single table:

In [34]:
MIhra = pd.DataFrame(index = MIhra5.index, columns =["MI-HRA1","MI-HRA2", "MI-HRA3", "MI-HRA4", "MI-HRA5"], data = pd.concat([MIhra1,MIhra2,MIhra3,MIhra4,MIhra5], axis = 1) .values )
MIhra

,MI-HRA1,MI-HRA2,MI-HRA3,MI-HRA4,MI-HRA5
cardeiranatural,0.000324,0.073983,0.333333,1.0,0.0


### Computation of the Global index (HMI-HRA):

In [35]:
#Weights (wis) of each index. They represent the number of indicatores per group:
wistable = [12,4,2,2,2]

# Now we proceed with a matrix multiplication:
wisarraey = np.array(wistable)
mihrasarray = np.array(MIhra.values)
# Finally one DataFrame with the Global indexes is computed:
GMIs = pd.DataFrame(index =MIhra5.index, columns = ["GMI-HRA"], data = (np.dot(mihrasarray,np.transpose(wisarraey)))/(22))
GMIs

,GMI-HRA
cardeiranatural,0.13484
